In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
#from drugbank_downloader import download_drugbank, open_drugbank
import repos_tools
import proximity
import pandas as pd
import matplotlib.pyplot as plt
import attila_utils
from toolbox import wrappers
from toolbox import network_utilities

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


In [2]:
fpath = '../../results/2021-08-11-drugbank/drugbank-filtered-proteins.csv'
drugbank_prot = pd.read_csv(fpath, index_col=(0, 1), dtype={'entrez_id': 'str'})
drugbank_prot

symbol  \
drugbank_id uniprot_id          
DB00006     P00734         F2   
DB00007     P30968      GNRHR   
DB00014     P22888      LHCGR   
            P30968      GNRHR   
DB00035     P30518      AVPR2   
...                       ...   
DB15822     P21802      FGFR2   
DB15873     P30874      SSTR2   
DB15982     P03952      KLKB1   
DB16019     P07288       KLK3   
DB16353     P23467      PTPRB   

                                                                   name  \
drugbank_id uniprot_id                                                    
DB00006     P00734                                          Prothrombin   
DB00007     P30968              Gonadotropin-releasing hormone receptor   
DB00014     P22888         Lutropin-choriogonadotropic hormone receptor   
            P30968              Gonadotropin-releasing hormone receptor   
DB00035     P30518                              Vasopressin V2 receptor   
...                                                                 ...   
DB15822     P21802                  Fibroblast growth factor receptor 2   
DB15873     P30874                         Somatostatin receptor type 2   
DB15982     P03952                                    Plasma kallikrein   
DB16019     P07288                            Prostate-specific antigen   
DB16353     P23467      Receptor-type tyrosine-protein phosphatase beta   

                       category organism known_action    actions     hgnc_id  \
drugbank_id uniprot_id                                                         
DB00006     P00734       target   Humans          yes  inhibitor   HGNC:3535   
DB00007     P30968       target   Humans          yes    agonist   HGNC:4421   
DB00014     P22888       target   Humans          yes    agonist   HGNC:6585   
            P30968       target   Humans          yes    agonist   HGNC:4421   
DB00035     P30518       target   Humans          yes    agonist    HGNC:897   
...                         ...      ...          ...        ...         ...   
DB15822     P21802       target   Humans      unknown  inhibitor   HGNC:3689   
DB15873     P30874       target   Humans          yes     binder  HGNC:11331   
DB15982     P03952       target   Humans          yes  inhibitor   HGNC:6371   
DB16019     P07288       target   Humans          yes     binder   HGNC:6364   
DB16353     P23467       target   Humans          yes  inhibitor   HGNC:9665   

                                  group entrez_id  
drugbank_id uniprot_id                             
DB00006     P00734             approved      2147  
DB00007     P30968             approved      2798  
DB00014     P22888             approved      3973  
            P30968             approved      2798  
DB00035     P30518             approved       554  
...                                 ...       ...  
DB15822     P21802             approved      2263  
DB15873     P30874             approved      6752  
DB15982     P03952             approved      3818  
DB16019     P07288             approved       354  
DB16353     P23467      investigational      5787  

[12999 rows x 9 columns]

In [3]:
drugbank_prot_small = drugbank_prot.iloc[0:9]
val = proximity.calculate_proximities(drugbank_prot_small, dis_genes_name='knowledge', asynchronous=True)

{'MT-ND1', 'MT-ND2'}
DB00006 processed in 9.3s
DB00007 processed in 11.8s
DB00050 processed in 17.2s
DB00014 processed in 19.3s
DB00035 processed in 26.4s
total runtime: 27.7s


In [4]:
val1 = proximity.calculate_proximities(drugbank_prot_small, dis_genes_name='knowledge', asynchronous=False)

{'MT-ND1', 'MT-ND2'}
DB00006 processed in 6.7s
DB00007 processed in 9.9s
DB00014 processed in 13.7s
DB00035 processed in 21.4s
DB00050 processed in 14.2s
total runtime: 67.2s


In [5]:
val

,name,type,groups,symbol,hgnc_id,d,avg_d_H0,sdev_d_H0,z,p
drugbank_id,,,,,,,,,,
DB00006,Bivalirudin,small molecule,approved|investigational,F2,HGNC:3535,2.0,1.726000,0.461437,0.593797,0.723676
DB00007,Leuprolide,small molecule,approved|investigational,GNRHR,HGNC:4421,2.0,2.143000,0.443341,-0.322551,0.373518
DB00014,Goserelin,small molecule,approved,LHCGR|GNRHR,HGNC:6585|HGNC:4421,2.0,2.013500,0.246105,-0.054855,0.478127
DB00035,Desmopressin,small molecule,approved,AVPR2|AVPR1A|AVPR1B,HGNC:897|HGNC:895|HGNC:896,2.0,2.047667,0.238642,-0.199741,0.420842
DB00050,Cetrorelix,small molecule,approved|investigational,GNRHR|LHCGR,HGNC:4421|HGNC:6585,2.0,2.013500,0.246105,-0.054855,0.478127


In [6]:
%connect_info

{
  "shell_port": 51207,
  "iopub_port": 58779,
  "stdin_port": 45309,
  "control_port": 51435,
  "hb_port": 41287,
  "ip": "127.0.0.1",
  "key": "87bb1421-3f571beeff3396f6dda3b7b6",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-f23047ed-40af-4d42-a375-82f05529f57a.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
